In [1]:
import torch
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from PIL import Image
import numpy as np
import cv2
import os

error_range = 40

In [2]:
def extract_coordinates():
    model = fasterrcnn_resnet50_fpn(pretrained=True) 
    model.eval()

    folder_path = "processed-data"
    image_files = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith('.jpg')]

    transform = T.Compose([T.ToTensor()])
    image_inputs = []

    for image_file in image_files:
        image = Image.open(image_file)
        image_tensor = transform(image)
        image_tensor = image_tensor.unsqueeze(0)  

        with torch.no_grad():
            prediction = model(image_tensor)

        boxes = prediction[0]['boxes'].cpu().numpy()
        labels = prediction[0]['labels'].cpu().numpy()
        car_coordinates = []

        for i in range(len(boxes)):
            if labels[i] == 3 or labels[i] == 6: # 3: car, 6: bus
                x, y, w, h = boxes[i]
                x_center = x + w/2
                y_center = y + h/2
                car_coordinates.append((x, y, w, h))

        image_inputs.append([tuple(coord[:2]) for coord in car_coordinates])

    for i, car_coordinates in enumerate(image_inputs):
        print(f"Car coordinates in img{i + 1}.jpg: {car_coordinates}")
    return image_inputs

In [3]:
parking_coordinates = []
nonparking_coordinates = []
image_inputs = extract_coordinates()

/Users/aruzhan/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/aruzhan/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Car coordinates in img1.jpg: [(2054.0288, 870.5265), (2836.7253, 520.2079), (2415.1428, 581.59515), (4243.632, 1328.2269), (1699.827, 1047.4218), (3856.973, 1465.8416), (3420.3745, 211.53253), (4055.9482, 1422.2593), (1049.3441, 1223.3123), (3548.0337, 816.37024)]
Car coordinates in img2.jpg: [(2137.1091, 865.92084), (1955.648, 1936.8843), (2480.7883, 578.89716), (2923.4575, 504.76205), (1787.2633, 1044.6377), (3486.9458, 208.89304), (1179.7832, 1218.4658), (1160.5503, 1246.4061)]
Car coordinates in img3.jpg: [(4219.421, 1337.3335), (2055.1604, 872.42316), (2384.526, 1736.9271), (2406.0261, 572.62506), (2838.0735, 516.54425), (1701.7076, 1064.7874), (3410.6055, 215.3643), (1068.9104, 1246.564)]
Car coordinates in img4.jpg: [(2078.4854, 861.7798), (4226.349, 1323.3093), (2858.74, 503.55185), (2434.1567, 553.93646), (1699.3206, 1053.6636), (3896.3064, 1444.4288), (651.9003, 1459.3466), (2670.279, 2239.3474), (2033.4235, 1626.2002), (2132.2205, 2407.2437), (1085.1926, 1226.4908), (3547.67

In [6]:
sorted_inputs = [sorted(group, key=lambda coord: coord[0]) for group in image_inputs]
print(sorted_inputs[:2])

[[(1049.3441, 1223.3123), (1699.827, 1047.4218), (2054.0288, 870.5265), (2415.1428, 581.59515), (2836.7253, 520.2079), (3420.3745, 211.53253), (3548.0337, 816.37024), (3856.973, 1465.8416), (4055.9482, 1422.2593), (4243.632, 1328.2269)], [(1160.5503, 1246.4061), (1179.7832, 1218.4658), (1787.2633, 1044.6377), (1955.648, 1936.8843), (2137.1091, 865.92084), (2480.7883, 578.89716), (2923.4575, 504.76205), (3486.9458, 208.89304)]]


In [4]:
def isSameArea(coord1, coord2):
    x_diff = abs(coord1[0] - coord2[0])
    y_diff = abs(coord1[1] - coord2[1])
    if x_diff <= error_range and y_diff <= error_range:
        return True
    else:
        return False

In [10]:
sorted_inputs = sorted_inputs[:207]
print(len(sorted_inputs))

207


In [11]:
for i in range(0, len(sorted_inputs), 3):
        group = sorted_inputs[i:i+3]
        for coord in group[0]:
            added = False
            for coord2 in group[1]:
                if(isSameArea(coord, coord2)):
                    parking_coordinates.extend([coord, coord2])
                    for coord3 in group[2]:
                        if(isSameArea(coord, coord3) and isSameArea(coord2, coord3)): 
                            parking_coordinates.extend([coord, coord2, coord3])
                            added = True
                        else:  
                            if(coord3 not in parking_coordinates):
                                nonparking_coordinates.extend([coord3])
                else:
                    if(coord2 not in parking_coordinates):
                        nonparking_coordinates.extend([coord2])
                    for coord3 in group[2]:
                        if(isSameArea(coord2, coord3)):
                            parking_coordinates.extend([coord2, coord3])
            if(added == False and coord not in parking_coordinates):
                nonparking_coordinates.extend([coord])

In [12]:
parking_set = set(parking_coordinates)
nonparking_coordinates = [coord for coord in nonparking_coordinates if coord not in parking_set]
parking_coordinates = list(parking_set)
nonparking_coordinates = list(set(nonparking_coordinates))

In [13]:
print("\nParking Coordinates:", parking_coordinates[:10])
print("Non-Parking Coordinates:", nonparking_coordinates[:10])


Parking Coordinates: [(3425.8562, 201.83078), (2645.0989, 2250.35), (2668.8413, 2246.9314), (2046.3309, 853.2959), (4218.9136, 1332.1956), (3855.269, 1408.9409), (4212.6367, 1336.3967), (2059.72, 836.0832), (2427.4216, 538.9845), (2411.0862, 559.62714)]
Non-Parking Coordinates: [(3606.9265, 845.989), (4705.246, 860.5909), (665.23975, 1455.0223), (3159.1055, 2009.902), (3855.1555, 1436.9237), (1144.5533, 1198.5133), (2053.5706, 2334.896), (2274.3313, 2799.9556), (2739.2961, 532.7024), (3893.4136, 1213.5826)]


In [14]:
image_array = cv2.imread("test.jpg")
for coord in parking_coordinates:
        x, y = coord
        x, y = int(x), int(y)
        width, height = 500, 500  
        cv2.rectangle(image_array, (x, y), (x + width, y + height), (0, 255, 0), 2)  

for coord in nonparking_coordinates:
    x, y = coord
    x, y = int(x), int(y)
    width, height = 500, 500  
    # radius = 50
    cv2.rectangle(image_array, (x, y), (x + width, y + height), (0, 0, 255), 2)  
    # cv2.circle(image_array, (x, y), radius, (0, 0, 255), 2)

cv2.imshow("Parking Coordinates", image_array)
cv2.waitKey(0)
cv2.destroyAllWindows()

Corrupt JPEG data: premature end of data segment
